In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('Data/insurance_fraud_data.csv')

print(f"Initial shape: {df.shape}")
df.head()

Initial shape: (12002, 29)


,claim_number,age_of_driver,gender,marital_status,safety_rating,annual_income,high_education,address_change,property_status,zip_code,...,vehicle_category,vehicle_price,vehicle_color,total_claim,injury_claim,policy deductible,annual premium,days open,form defects,fraud reported
0,414724,39,M,1,73,58612.8,1,0,Own,50048,...,Large,24360.59273,silver,26633.27819,5196.552552,1000,1406.91,8.641800,5,N
1,269568,33,M,1,72,35936.0,0,1,Own,50006,...,Medium,23457.35282,black,26707.46021,7957.267641,2000,1415.74,10.009761,5,Y
2,974592,31,M,1,76,84940.8,1,1,Own,15021,...,Medium,19752.42523,gray,15279.38583,3014.110884,1000,1199.44,8.264032,3,N
3,995328,53,F,1,93,73526.4,0,1,Own,85027,...,Medium,32910.34224,red,20909.27772,5671.599025,500,708.64,9.235943,4,N
4,1140480,41,M,1,87,59403.2,1,0,Own,80046,...,Medium,28570.37675,red,34493.97997,4064.569489,1000,1187.96,10.259629,2,N


In [4]:
# Global Replacement of noise characters
df.replace(['*', '?', 'NA'], np.nan, inplace=True)

In [5]:
# List of all columns that should be strictly 0 or 1
binary_columns = [
    'marital_status', 
    'high_education', 
    'address_change', 
    'witness_present', 
    'police_report'
]

for col in binary_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    
    col_mode = df[col].mode()[0]
    df[col] = df[col].fillna(col_mode).astype(int)

In [6]:
#  Standardize text: Remove spaces and make uppercase (M, F)
df['gender'] = df['gender'].str.strip().str.upper()

#  Map to full words for better Dashboard labels
df['gender'] = df['gender'].map({'M': 'Male', 'F': 'Female'})

#  Handle missing values: 
gender_mode = df['gender'].mode()[0]
df['gender'] = df['gender'].fillna(gender_mode)

In [ ]:
df['fraud_reported'] = (
    df['fraud reported']
    .str.strip()          
    .str.upper()          
    .map({'Y': 1, 'N': 0})
)

# Fill unexpected/missing values with the mode 
fraud_mode = df['fraud_reported'].mode()[0]
df['fraud_reported'] = df['fraud_reported'].fillna(fraud_mode).astype(int)

# Drop original column
df = df.drop(columns=['fraud reported'])

In [8]:
# General Numeric Cleaning & Imputation
numeric_cols = ['annual_income', 'policy deductible', 'annual premium', 
                'liab_prct', 'past_num_of_claims', 'age_of_driver', 
                'total_claim', 'injury_claim', 'vehicle_price']

for col in numeric_cols:
    if df[col].dtype == 'object':
        df[col] = df[col].astype(str).str.replace(r'[$,]', '', regex=True)
    df[col] = pd.to_numeric(df[col], errors='coerce')
    if df[col].isnull().sum() > 0:
        df[col] = df[col].fillna(df[col].median())

In [9]:
# LOGICAL FIXES & DATES
df['annual_income'] = df['annual_income'].abs()
df['age_of_driver'] = df['age_of_driver'].abs().clip(18, 100)
df['claim_date'] = pd.to_datetime(df['claim_date'], errors='coerce').ffill()
df['claim_year'] = df['claim_date'].dt.year
df['claim_month'] = df['claim_date'].dt.month_name()

In [10]:
df.describe()

,claim_number,age_of_driver,marital_status,safety_rating,annual_income,high_education,address_change,zip_code,claim_date,past_num_of_claims,...,police_report,vehicle_price,total_claim,injury_claim,policy deductible,annual premium,days open,form defects,fraud_reported,claim_year
count,1.200200e+04,12002.000000,12002.000000,12002.000000,12002.000000,12002.000000,12002.000000,12002.000000,12002,12002.000000,...,12002.000000,12002.000000,12002.000000,12002.000000,12002.000000,12002.000000,12002.000000,12002.000000,12002.000000,12002.000000
mean,3.119801e+08,43.489252,0.715214,73.702966,63658.228567,0.698134,0.576737,49762.387269,2023-12-29 18:31:29.685052416,0.494918,...,0.601066,22925.335594,22861.534958,5037.353960,1011.039827,1268.140663,8.799198,3.969588,0.245876,2023.492418
min,1.866240e+05,19.000000,0.000000,2.000000,1.000000,0.000000,0.000000,0.000000,2023-01-01 00:00:00,0.000000,...,0.000000,2261.507903,2583.428985,471.363743,500.000000,433.330000,2.303305,0.000000,0.000000,2023.000000
25%,1.561887e+08,35.000000,0.000000,66.000000,56899.200000,0.000000,0.000000,20111.000000,2023-06-26 00:00:00,0.000000,...,0.000000,14087.559950,13832.784382,3373.309069,500.000000,1082.360000,8.025550,3.000000,0.000000,2023.000000
50%,3.121908e+08,43.000000,1.000000,76.000000,60899.200000,1.000000,1.000000,50027.000000,2023-12-26 00:00:00,0.000000,...,1.000000,20642.029325,20508.293695,4678.122607,1000.000000,1239.060000,9.268787,4.000000,0.000000,2023.000000
75%,4.677938e+08,51.000000,1.000000,85.000000,64697.600000,1.000000,1.000000,80036.750000,2024-07-01 00:00:00,1.000000,...,1.000000,29293.641235,29205.553660,6305.066147,1000.000000,1383.490000,10.307928,5.000000,0.000000,2024.000000
max,6.220385e+08,100.000000,1.000000,100.000000,257313.600000,1.000000,1.000000,85083.000000,2024-12-31 00:00:00,6.000000,...,1.000000,106991.684500,101818.964200,29763.669890,2000.000000,2047.590000,15.177169,13.000000,1.000000,2024.000000
std,1.795997e+08,11.821440,0.451331,15.378967,17289.697073,0.459087,0.494097,29133.180765,NaN,0.956002,...,0.489699,12108.888412,12197.334761,2316.870095,562.099581,231.670671,2.413533,1.986915,0.430623,0.499963


In [12]:
df.columns = df.columns.str.replace(' ', '_').str.lower()
df.to_csv('Data/insurance_fraud_data_cleaned.csv', index=False)

In [ ]:
# Display unique values for each column in the dataset to verify cleaning
for column in df.columns:
    unique_values = df[column].unique()
    print(f"Column: {column}")
    print(f"Unique Values: {unique_values}")
    print("-")

Column: claim_number
Unique Values: [   414724    269568    974592 ... 621872640 621914112 621955584]
-
Column: age_of_driver
Unique Values: [ 39  33  31  53  41  42  50  30  49  70  28  38  23  55  44  37  40  35
  54  27  62  34  47  32  46  57  56  51  45  52  58  63  48  80  43  65
  68  26  77  24  22  29  59  60  36  61  73  25  64  81  86  75  66  21
  69  71  67  74  79  82  78  96  72  76  19  93  88  85  20  89  83  84
 100  87  99  92]
-
Column: gender
Unique Values: ['Male' 'Female']
-
Column: marital_status
Unique Values: [1 0]
-
Column: safety_rating
Unique Values: [ 73  72  76  93  87  68  89  79  57  58  49  78  92  82  83  65  77  63
  36  90  94  44  75  45  66  64  70  86  74  80  32  62  69  88  84  41
  91  30  52  50  71  85  59  98  81  60  42  67  55  95  51   7  35  54
  61  17  56  43  53  10  99  40  38  47  26  20  96  33  34  31  37  39
  18  16  97  21  46  27  22  25  29  15  48  28  24  13  23 100  12   9
   8  19  11   4   5  14   2   3]
-
Column: annua